In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
palette = sns.color_palette('bright')
palette

[(0.00784313725490196, 0.24313725490196078, 1.0),
 (1.0, 0.48627450980392156, 0.0),
 (0.10196078431372549, 0.788235294117647, 0.2196078431372549),
 (0.9098039215686274, 0.0, 0.043137254901960784),
 (0.5450980392156862, 0.16862745098039217, 0.8862745098039215),
 (0.6235294117647059, 0.2823529411764706, 0.0),
 (0.9450980392156862, 0.2980392156862745, 0.7568627450980392),
 (0.6392156862745098, 0.6392156862745098, 0.6392156862745098),
 (1.0, 0.7686274509803922, 0.0),
 (0.0, 0.8431372549019608, 1.0)]

In [ ]:
solar_data = pd.DataFrame(pd.read_csv('C:/Users/mhkim/OneDrive/문서/GitHub/Solar_Nbody_system/solarsystem.csv', sep=','))
solar_data.set_index('name',inplace=True,drop=True)
solar_data.head()

,distance,velocity,mu
name,,,
sun,0,0,132712440018000000000
mercury,58344000000,47400,22032000000000
venus,108160800000,35100,324859000000000
earth,149600000000,29800,398600441800000
mars,227990400000,24100,42828370000000


In [ ]:
#object class part

class Planet:

    def __init__ (self,PlanetName):
        __solar_data = pd.DataFrame(pd.read_csv('C:/Users/mhkim/OneDrive/문서/GitHub/Solar_Nbody_system/solarsystem.csv', sep=','))
        __solar_data.set_index('name',inplace=True,drop=True)    
        __pos = np.array([0,0],float)
        __vel = np.array([0,0],float)
        self.PlanetName = str(PlanetName)
        self.position = __pos+np.array([__solar_data.loc[self.PlanetName][0],0],float) #Nowstate position
        self.velocity = __vel+np.array([__solar_data.loc[self.PlanetName][1],0],float) #Nowstate velocity
        self.mu = __solar_data.loc[self.PlanetName][2]
        self.distance = __solar_data.loc[self.PlanetName][0] #Distance from the Sun 
        
    def update (self, new_position, new_velocity):      
        self.position += new_position
        self.velocity += new_velocity

    def info(self):
        print('Planet Name:',self.PlanetName)
        print('Distance From Sun:',self.distance)
        print('Now Coordinate:',self.position)
        print('Now Velocity:',self.velocity)
        print('Planet standard gravitational parameter (mu):',self.mu)

    def __str__(self):
        return ("This is '{0}' ".format(self.PlanetName))

class Solarsystem(list):
    def __init__(self):
        super().__init__
        self.extend(Planet(name) for name in solar_data.index )

In [ ]:
#Calculation function part

def f(data,indexnumber):
    coord = np.split(data,2)[0]
    v = np.split(data,2)[1]
    dummy = Solarsystem()
    del dummy[indexnumber]
    #next position
    fr = v
    #next velocity 
    fv = np.array([0,0],float)
    for z in range(0,len(dummy)):
        dist = (np.abs(np.sqrt(np.sum(dummy[z].position**2)) - np.sqrt(np.sum(coord**2)))) 
        fv += float(dummy[z].mu)*coord/(dist**3)                    
        
    answer = np.concatenate((fr,fv),axis=None,dtype=float)
    return answer

def update(function,h,k):
    data =  np.concatenate((function.position,function.velocity),axis=None, dtype=float)     
    k1 = h * f(data,k)
    k2 = h * f(data+k1/2,k)
    k3 = h * f(data+k2/2,k)
    k4 = h * f(data+k3,k)
    newdata = data + (k1+(2*k2)+(2*k3)+k4)/6
    function.update(np.split(newdata,2)[0],np.split(newdata,2)[1])
    

    


In [ ]:
#Main Part
if __name__ == '__main__':
    fig = plt.figure(figsize=(15,15))
    ax = fig.add_subplot(1, 1, 1)


    solsy = Solarsystem()       #call solarsystem

    t0 = 0
    t_end =  31600000             #initial time setting
    dt = 1.3e4                  #step size
    end = int(t_end/dt)+1
     
    t = t0
    for i in range(end-1):
        for k in range(1,len(solsy)):
            plt.scatter(solsy[k].position[0],solsy[k].position[1])
            update(solsy[k],dt,k)
            
        t += dt


    legend_properties = {'weight':'bold','size':'10'}       
    planet_names = ['Mercury','Venus','Earth','Mars','Jupiter','Saturn','Uranus','Neptune','Pluto','Halley']
    ax.set_xlabel('x location(AU)',fontsize = 14, fontweight='bold',loc='center')
    ax.set_ylabel('y location(AU)',rotation=90, fontsize = 14, fontweight='bold',loc='center')
    
    plt.legend(loc=1,frameon=False, prop=legend_properties)
    plt.show()

    

In [150]:
solsy[10].info()

Planet Name: halley
Distance From Sun: 5280880000000
Now Coordinate: [5.38065083e+102 0.00000000e+000]
Now Velocity: [1.01855459e+93 0.00000000e+00]
Planet standard gravitational parameter (mu): 14674 
